# Введение
---

**Цель анализа**: провести первичную оценку качества набора данных о физико-химических свойствах веществ, выявить основные проблемы в данных и подготовить их к дальнейшему анализу.

**Источник:** [Kaggle — Physical and Chemical Properties of Substances](https://www.kaggle.com/datasets/ivanyakovlevg/physical-and-chemical-properties-of-substances/data)

**Содержание:**

Набор данных содержит информацию о различных химических веществах, включая их физические и термодинамические свойства.
Каждая запись соответствует одному веществу и включает следующие параметры:

* Химическая формула и название
* Молекулярная масса
* Температуры плавления и кипения
* Плотность
* Теплоёмкость, энтальпия, энтропия
* Критическая температура и давление
* Другие физико-химические параметры, используемые в термодинамическом моделировании и вычислительной химии

Этот датасет подходит для:
* Оценки качества и полноты данных
* Анализа корреляций и признаков
* Построения предсказательных моделей для оценки неизвестных свойств веществ

План работы:
1. Загрузка и первичный осмотр
2. Оценка полноты данных
3. Анализ выбросов и аномалий
4. Итоговые выводы

## Загрузка данных и первичный осмотр ##

Используем скрипт ```data_loader.py``` для загрузки данных и преобразовании в ```parquet```

In [9]:
%run ../src/data_loader.py
df = pd.read_parquet('clean_data.parquet')

Файл уже есть, читаю локальную копию.
name                     object
formula                  object
CAS                      object
smiles                   object
InChI                    object
InChIKey                 object
molecular_weight        float64
melting_point_K         float64
boiling_point_K         float64
heat_of_fusion          float64
heat_of_vaporization    float64
critical_temperature    float64
critical_pressure       float64
flash_point             float64
logP                    float64
dtype: object
                                 name        formula        CAS  \
0                             ammonia            H3N  7664-41-7   
1  1,4-benzodioxane-2-carboxylic acid         C9H8O4  3663-80-7   
2                           acetylene           C2H2    74-86-2   
3              adenosine triphosphate  C10H16N5O13P3    56-65-5   
4                     rhodizonic acid         C6H2O6   118-76-3   

                                              smiles  \
0        

Данные успешно загружены и преобразованы в ```parquet```. Командами ```raw_data.dtypes.head(15)``` и
```raw_data.head(5)``` выведены первые столбцы и строки и типы данных.

## Оценка целостности и полноты данных

Проверим данные на наличие пропущенных значений и полных дубликатов. Это критически важный шаг для оценки качества данных.

In [11]:
# Считаем количество пропусков в каждом столбце
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Создаем DataFrame для наглядного отображения пропусков
missing_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Percentage (%)', ascending=False)

print("Информация о пропущенных значениях:")
display(missing_info)

# Проверяем наличие полных дубликатов
num_duplicates = df.duplicated().sum()
print(f"\nНайдено полных дубликатов строк: {num_duplicates}")

Информация о пропущенных значениях:


,Missing Values,Percentage (%)
flash_point,4070,93.714023
logP,3634,83.674879
heat_of_vaporization,2520,58.024407
heat_of_fusion,987,22.726226
critical_pressure,842,19.387520
critical_temperature,824,18.973060
boiling_point_K,709,16.325121
improved_name,48,1.105227
class,25,0.575639
smiles,12,0.276307



Найдено полных дубликатов строк: 0


### Выводы и метрики по целостности

* **Метрика: Количество столбцов с пропусками:** ```15```
* **Метрика: Процент пропусков для самого проблемного столбца:** ```93.7```
* **Метрика: Количество дубликатов:** `0`

**Вывод:** анализ показал наличие пропусков в нескольких столбцах. Наиболее проблемным является столбец `flash_point`, где отсутствует `93.7%` данных. Это может потребовать либо удаления строк, либо использования стратегий для заполнения пропусков. Полных дубликатов обнаружено не было.

## Оценка выбросов и аномалий

Изучим распределения данных, чтобы найти значения, которые сильно выделяются из общей картины. Для этого воспользуемся описательными статистиками для числовых признаков и подсчетом значений для булевых.

In [12]:
# Получаем описательные статистики для всех числовых столбцов
# 'display()' используется для красивого вывода в Jupyter
print("Описательные статистики для числовых признаков:")
display(df.describe())

Описательные статистики для числовых признаков:


,molecular_weight,melting_point_K,boiling_point_K,heat_of_fusion,heat_of_vaporization,critical_temperature,critical_pressure,flash_point,logP
count,4343.000000,4343.000000,3634.000000,3.356000e+03,1.823000e+03,3519.000000,3.501000e+03,273.000000,709.000000
mean,269.160696,514.716704,811.599397,1.424487e+05,4.532314e+05,1106.093084,5.595430e+06,338.622960,2.097278
std,179.075599,400.066559,546.277999,1.318577e+05,1.331007e+06,4384.442863,3.378248e+07,69.082691,2.175028
min,1.007940,0.950000,4.223807,2.554609e+02,0.000000e+00,-81884.985000,1.062812e+05,185.751749,-3.690000
25%,157.336690,337.265000,502.425000,1.048341e+05,2.175428e+05,750.140760,1.916937e+06,285.150000,0.510000
50%,244.285740,432.150000,715.830000,1.299734e+05,2.968804e+05,966.760120,2.826327e+06,334.150000,2.050000
75%,337.411450,530.150000,935.177500,1.560281e+05,4.088440e+05,1183.086050,4.360679e+06,383.150000,3.610000
max,3354.070500,7449.440000,8574.340000,3.438407e+06,3.584615e+07,222858.230000,8.421600e+08,549.816670,9.360000


#### Выводы по числовым признакам

Анализ описательных статистик позволяет сделать следующие наблюдения:

* **Столбец `molecular_weight`:** среднее значение ([mean]) близко к медиане ([50%]), что говорит о достаточно симметричном распределении. Минимальное и максимальное значения выглядят правдоподобно.
* **Столбец `critical_temperature`:** присутствуют сильно отрицательные значения в `min`, что физически невозможно для температуры. Это явная аномалия или ошибка в данных.

#### Анализ булевых и категориальных признаков

In [ ]:
# Анализируем распределение булевых признаков
bool_cols = [c for c in df.columns if c.startswith("is_")]
for col in bool_cols:
    print(f"Распределение для столбца '{col}':")
    print(df[col].value_counts(normalize=True)) # normalize=True показывает долю

# Проверяем уникальность идентификатора CAS 
print(f"Всего записей: {len(df)}")
print(f"Уникальных CAS: {df['CAS'].nunique()}")
if len(df) == df['CAS'].nunique():
    print("Идентификатор 'CAS' является уникальным для каждой строки.")
else:
    print("В 'CAS' есть дублирующиеся значения!")

Распределение для столбца 'is_organic':
is_organic
True     0.825927
False    0.174073
Name: proportion, dtype: float64
Распределение для столбца 'is_radionuclide':
is_radionuclide
False    1.0
Name: proportion, dtype: float64
Распределение для столбца 'is_hydrocarbon':
is_hydrocarbon
False    0.968685
True     0.031315
Name: proportion, dtype: float64
Распределение для столбца 'is_alkane':
is_alkane
False    0.990099
True     0.009901
Name: proportion, dtype: float64
Распределение для столбца 'is_cycloalkane':
is_cycloalkane
False    0.997007
True     0.002993
Name: proportion, dtype: float64
Распределение для столбца 'is_branched_alkane':
is_branched_alkane
False    0.999309
True     0.000691
Name: proportion, dtype: float64
Распределение для столбца 'is_alkene':
is_alkene
False    0.811651
True     0.188349
Name: proportion, dtype: float64
Распределение для столбца 'is_alkyne':
is_alkyne
False    0.989638
True     0.010362
Name: proportion, dtype: float64
Распределение для столбца '

#### Выводы по булевым и категориальным признакам

* **Булевы признаки:** в данных преобладают органические соединения ([83]%). Радионуклиды отсутствуют. Значительная часть веществ - углеводороды.
* **Идентификатор `cas_number`:** проверка показала, что значения CAS дублируются, что может быть ошибкой в данных и требует правки/удаления.

## Заключение
---
Проведенный разведочный анализ данных позволил составить представление о качестве и структуре предоставленного набора данных

**Основные выводы:**
1.  **Структура:** данные имеют четкую структуру, типы данных в основном корректны после первичной обработки.
2.  **Полнота:** обнаружены пропуски в ключевых числовых столбцах (например, `flash point`), что является основной проблемой целостности данных.
3.  **Корректность:** выявлены явные аномалии в данных, такие как сильно отрицательные значения температуры (`critical_temperature`), что указывает на ошибки при сборе или обработке данных.
